# Question-Answering
- Data : Stanford Question Answering Dataset (SQuAD)

- 일련의 Wikipedia 기사에서 크라우드 워커가 제기 한 질문으로 구성된 독해 이해 데이터 세트


- task의 목표 : 주어진 질문에 대한 단락에서 답을 추출하는 것

- input : 질문(Q)-단락(paragraph) 쌍(question-paragraph pair)

- Q : 면역 체계는 무엇입니까?
- paragraph : 면역 체계는 질병으로부터 보호하는 유기체 내의 다양한 생물학적 구조와 과정의 시스템입니다. 제대로 기능하려면 .... 합니다.

- output : 단락에서 응답에 해당하는 텍스트 범위(시작/끝 index)
- A : 질병으로부터 보호하는 유기체 내의 다양한 생물학적 구조와 과정의 시스템입니다.

In [ ]:
!pip install transformers

In [3]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

### 사용할 모델 

'bert-large-uncased-whole-word-masking-finetuned-squad' : 스탠포드 질문-응답 데이터셋(SQUAD)을 기반으로 파인튜닝된 모델

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad') 
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
question = "What is the immune system?" # "면역 체계는 무엇입니까?"
paragraph = "The immune system is a system of various biological structures and processes within an organism that protects against disease. To function properly, the immune system must detect a variety of substances known as pathogens, from viruses to parasites, and distinguish them from the healthy tissue of an organism." # "면역 체계는 질병으로부터 보호하는 유기체 내의 다양한 생물학적 구조와 과정의 시스템입니다. 제대로 기능하려면 면역 체계가 바이러스에서 기생충에 이르기까지 병원균으로 알려진 다양한 물질을 탐지하고 유기체의 건강한 조직과 구별해야 합니다."

In [6]:
# bert input 맞춰주기
question = '[CLS]' + question + '[SEP]' 
paragraph = paragraph + '[SEP]' 

In [ ]:
question_tokens = tokenizer.tokenize(question)
paragraph_tokens = tokenizer.tokenize(paragraph)
tokens = question_tokens + paragraph_tokens 
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(input_ids)

In [ ]:
segment_ids = [0] * len(question_tokens)
segment_ids += [1] * len(paragraph_tokens)
print(segment_ids)
len(input_ids), len(segment_ids)

In [ ]:
tokenizer.decode(torch.tensor(input_ids)) # bert input 확인

In [ ]:
 # bert input tensor로 변환
input_ids = torch.tensor([input_ids])
segment_ids = torch.tensor([segment_ids]) 
input_ids

# 응답 얻기

In [ ]:
output = model(input_ids, token_type_ids = segment_ids) 
start_scores, end_scores = output['start_logits'], output['end_logits'] 
start_scores, end_scores

In [ ]:
start_index = torch.argmax(start_scores) # argmax: 가장 큰 값의 인덱스를 반환
end_index = torch.argmax(end_scores) 
start_index, end_index

In [ ]:
print(question[5:-5])
print(' '.join(tokens[start_index:end_index+1]))
# 질병으로부터 보호하는 유기체 내의 다양한 생물학적 구조와 과정의 시스템